In [ ]:
import os
import s3fs
import logging
import boto3
from botocore.client import Config


class S3_provider():
    """
    Класс для взаимодействия с хранилищем S3.

    Этот класс предоставляет методы для загрузки файлов из S3 хранилища. Он используется для загрузки и
    хранения моделей и данных, необходимых для работы сервиса.
    """
    def __init__(self):
        """
        Инициализация провайдера S3.

        Настраивает соединение с хранилищем S3, используя заданные параметры подключения.
        """
        # Кэш для моделей
        self.model_cache = {}

        # Настройки MinIO
        minio_access_key = "your_key"
        minio_secret_key = "your_key"
        minio_endpoint = "http://data-minio.udrvs-prod-aiplatform-tools-minio:9000"
        minio_bucket_name = "udrvs-prod-aiplatform-mlflow-dud-ds"

        self.s3 = boto3.resource('s3',
                            endpoint_url=minio_endpoint,
                            aws_access_key_id='your_key',
                            aws_secret_access_key='your_key',
                            config=Config(signature_version='s3v4'),
                            region_name='us-east-1')

        self.bucket_name = 'udrvs-prod-aiplatform-data-dud-ds'
        self.bucket = self.s3.Bucket(self.bucket_name)

        self.s3 = s3fs.S3FileSystem(endpoint_url=minio_endpoint,
                               key=minio_access_key,
                               secret=minio_secret_key)


    def download_from_s3(self, s3_folder="prod/vicuna_bot/model", local_folder='model', logging=None):
        """
        Загрузка файлов из S3 хранилища.

        Args:
            s3_folder (str): Путь к папке в S3.
            local_folder (str): Локальный путь для сохранения файлов.

        Загружает файлы из указанной папки S3 в локальную директорию. Создает локальные папки, если они
        не существуют, и логирует процесс загрузки.
        """
        if s3_folder in self.model_cache:
            return self.model_cache[s3_folder]  # Возвращаем кэшированную модель

        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - [%(levelname)s]: %(message)s",
            handlers=[
                logging.handlers.RotatingFileHandler(
                    filename="log.log",
                    mode="a",
                    maxBytes=1024,
                    backupCount=1,
                    encoding=None,
                    delay=0),
                logging.StreamHandler()
                ]
              )

        if not os.path.exists(local_folder):
            try:
                for obj in self.bucket.objects.filter(Prefix=s3_folder):
                    # Формирование пути для сохранения файла локально
                    local_path = os.path.join(local_folder, os.path.basename(obj.key))
                    # print(local_path)
                    # Создание локальной папки, если она не существует
                    os.makedirs(os.path.dirname(local_path), exist_ok=True)

                    # Загрузка файла из S3 в локальную папку
                    self.bucket.download_file(obj.key, local_path)

                    # Кэшируем загруженную модель
                    self.model_cache[s3_folder] = local_folder

                logging.info(f"Файлы успешно загружены из S3 в {local_path}")
            except Exception as e:
                logging.info(f"Ошибка при загрузке файла из S3: {str(e)}")
                return None
        return local_folder

    def threaded_function(self):
          """
          Функция для выполнения операций загрузки моделей в отдельном потоке.
          """
          from botocore.client import Config
          from threading import Thread
          import os

          self.s3_provider = self.s3_provider()
          # model_path = context.artifacts["s3_model_uri"]

          # self.s3_provider.download_from_s3(s3_folder="prod/vicuna_bot/instructor-xl", local_folder='instructor-xl')
          # self.s3_provider.download_from_s3(s3_folder="prod/vicuna_bot/instructor-xl/1_Pooling", local_folder='instructor-xl/1_Pooling')
          # self.s3_provider.download_from_s3(s3_folder="prod/vicuna_bot/instructor-xl/2_Dense", local_folder='instructor-xl/2_Dense')
          if not os.path.exists('LaBSE_ner_nerel'):
              self.s3_provider.download_from_s3(s3_folder='prod/vicuna_bot/LaBSE_ner_nerel', local_folder='LaBSE_ner_nerel')
          if not os.path.exists('sber_large_mt_nlu_ru'):
              self.s3_provider.download_from_s3(s3_folder='prod/vicuna_bot/sber_large_mt_nlu_ru', local_folder='sber_large_mt_nlu_ru')

          print('Мелкие модели загрузились')
          if not os.path.exists('model'):
              self.s3_provider.download_from_s3("prod/vicuna_bot/model", 'model')

          self.is_downloaded = True
          print('Модель загрузилась')